In [ ]:
#library management system

#define class'Book'
class Book:
    def __init__(self, bid, title, total, available):
        self.__bid = bid
        self.__title = title
        self.__total = total
        self.__available = available

    #define the function of borrowing book
    def borrow_book(self):
        try:
            if self.__available == 0:
                raise Exception('The library has ZERO inventory!')
            self.__available -= 1
            print(f'Borrowing book successfully, there is still {self.__available} book(s) available')
        except Exception as e:
            print(f'Borrowing failed: {e}!')

    #dedfine the function of returning book
    def return_book(self):
        try:
            if self.__available >= self.__total:
                raise Exception('The library inventory is full!')
            self.__available += 1
            print(f'Returning book successfully, there are {self.__available} book(s) available')
        except Exception as e:
            print(f'Returning failed: {e}!')

    #define the getter of bid
    def get_bid(self):
        return self.__bid


    #define the getter of title
    def get_title(self):
        return self.__title


    #define the getter of available
    def get_available(self):
        return self.__available

    #define the getter of total
    def get_total(self):
        return self.__total


#define class'User'
class User:
    borrowing_limit = 5

    def __init__(self, uid, username, borrowing_list):
        self.__uid = uid #学号/工号
        self.__username = username #姓名
        self.borrowing_list = borrowing_list #用户借书清单-->可更新

    def borrow_book(self, book, library):
        try:
            if len(self.borrowing_list) >= self.borrowing_limit:
                raise Exception('You have reached the borrowing limit of books!')
            elif book.get_available() == 0:
                raise Exception('The library has ZERO inventory!')

            book_id = book.get_bid()
            if library.has_book(book_id): #调用library类判断id方法，体现面向对象的封装原则
                self.borrowing_list.append(book_id) #更新借书清单
                book.borrow_book() #同步调用book类借书方法，更新available和total
                library.add_borrow_record(self.__username, book.get_title())
        except Exception as e:
            print(f'Borrowing failed: {e}!')

    def return_book(self, book, library):
        try:
            if not library.has_user(self.__uid):
                raise Exception('You have not registered a library account.') #主动抛出异常

            #设置两道还书保险
            book_id = book.get_bid()
            if not library.has_book(book_id): #不归还图书馆没有的书
                raise Exception('This book does not exist in the library.')
            if book_id not in self.borrowing_list: #不归还借书清单上不存在的书
                raise Exception('You have not borrowed this book from the library.')

            #移除相关记录并更新
            self.borrowing_list.remove(book_id) #移除对应借书
            book.return_book()
            library.remove_borrow_record(self.__username, book.get_title())
        except Exception as e:
            print(f'Returning failed: {e}!')

    def get_username(self):
        return self.__username


class Library:
    def __init__(self):

        #设置图书馆所有书籍及库存情况
        self.__books = {
            'A1001': Book('A1001', 'Stray Birds', 50, 40),
            'A1018': Book('A1018', 'Jane Eyre', 60, 30),
            'B3027': Book('B3027', 'Dragon Raja', 20, 0),
            'K1009': Book('K1009', 'One Hundred year of Solitude', 30, 1),
            'D2889': Book('D2889', 'the Holy Bible', 10, 10)
        }

        #设置图书馆在册用户及借书情况
        self.__users = {
            'U1001': User('U1001', 'Alice', ['A1001', 'A1018', 'B3027', 'K1009', 'D2889']),
            'U1002': User('U1002', 'Zhao', ['A1001', 'B3027']),
            'U1003': User('U1003', 'Mary', ['A1018', 'D2889']),
            'U1004': User('U1004', 'Mike', []),
            'U1005': User('U1005', 'Lee', [])
        }


        #设置借阅记录（以语言形式更直观）
        self.__borrow_records = [
            'Alice borrowed Stray Birds.',
            'Alice borrowed Jane Eyre.',
            'Alice borrowed Dragon Raja.',
            'Alice borrowed One Hundred year of Solitude.',
            'Alice borrowed the Holy Bible.',
            'Zhao borrowed Stray Birds.',
            'Zhao borrowed Dragon Raja.',
            'Mary borrowed Jane Eyre.',
            'Mary borrowed the Holy Bible.'
        ]

    #设置两个判断id是否在字典中的函数
    def has_book(self, book_id):
        return book_id in self.__books

    def has_user(self, user_id):
        return user_id in self.__users

    #设置借书还书记录更新函数
    def add_borrow_record(self, username, book_title):
        record = f'{username} borrowed {book_title}.'
        self.__borrow_records.append(record)

    def remove_borrow_record(self, username, book_title):
        for record in list(self.__borrow_records):
            if username in record and book_title in record:
                self.__borrow_records.remove(record)

    #getter定义
    def get_book(self, book_id):
        return self.__books.get(book_id)

    def get_user(self, user_id):
        return self.__users.get(user_id)

    def get_all_books(self):
        return list(self.__books.values())

    def get_all_users(self):
        return list(self.__users.values())


    #ui优化
    def show_all_books(self):
        print("\n=== All Books ===")
        for book_id, book in self.__books.items():
            print(f"ID: {book_id}, Title: {book.get_title()}, "
                  f"Available: {book.get_available()}/{book.get_total()}")

    def show_all_users(self):
        print("\n=== All Users ===")
        for user_id, user in self.__users.items():
            print(f"ID: {user_id}, Name: {user.get_username()}, "
                  f"Borrowed: {len(user.borrowing_list)} books")

    def show_borrow_records(self):
        print("\n=== Borrow Records ===")
        if not self.__borrow_records:
            print("No borrow records.")
        for record in self.__borrow_records:
            print(record)


def main():
    library = Library()

    #while True：让程序一直运行，直到用户主动选择退出，这是控制台交互程序最常用的写法
    while True:
        print("\n=== Library Management System ===")
        print("1. Show all books")
        print("2. Show all users")
        print("3. Show borrow records")
        print("4. Borrow a book")
        print("5. Return a book")
        print("6. Exit")

        choice = input("\nEnter your choice (1-6): ")
    
    #根据选择进行显示
        if choice == "1":
            library.show_all_books()
        elif choice == "2":
            library.show_all_users()
        elif choice == "3":
            library.show_borrow_records()
        elif choice == "4":
            user_id = input("Enter user ID: ")
            book_id = input("Enter book ID: ")
            user = library.get_user(user_id)
            book = library.get_book(book_id)
            if user and book:
                user.borrow_book(book, library)
            else:
                print("User or book not found!")
        elif choice == "5":
            user_id = input("Enter user ID: ")
            book_id = input("Enter book ID: ")
            user = library.get_user(user_id)
            book = library.get_book(book_id)
            if user and book:
                user.return_book(book, library)
            else:
                print("User or book not found!")
        elif choice == "6":
            print("Goodbye!")
            break
        else:
            print("Invalid choice! Please try again.")


if __name__ == "__main__":
    main()


=== Library Management System ===
1. Show all books
2. Show all users
3. Show borrow records
4. Borrow a book
5. Return a book
6. Exit



Enter your choice (1-6):  1



=== All Books ===
ID: A1001, Title: Stray Birds, Available: 40/50
ID: A1018, Title: Jane Eyre, Available: 30/60
ID: B3027, Title: Dragon Raja, Available: 0/20
ID: K1009, Title: One Hundred year of Solitude, Available: 1/30
ID: D2889, Title: the Holy Bible, Available: 10/10

=== Library Management System ===
1. Show all books
2. Show all users
3. Show borrow records
4. Borrow a book
5. Return a book
6. Exit



Enter your choice (1-6):  2



=== All Users ===
ID: U1001, Name: Alice, Borrowed: 5 books
ID: U1002, Name: Zhao, Borrowed: 2 books
ID: U1003, Name: Mary, Borrowed: 2 books
ID: U1004, Name: Mike, Borrowed: 0 books
ID: U1005, Name: Lee, Borrowed: 0 books

=== Library Management System ===
1. Show all books
2. Show all users
3. Show borrow records
4. Borrow a book
5. Return a book
6. Exit



Enter your choice (1-6):  3



=== Borrow Records ===
Alice borrowed Stray Birds.
Alice borrowed Jane Eyre.
Alice borrowed Dragon Raja.
Alice borrowed One Hundred year of Solitude.
Alice borrowed the Holy Bible.
Zhao borrowed Stray Birds.
Zhao borrowed Dragon Raja.
Mary borrowed Jane Eyre.
Mary borrowed the Holy Bible.

=== Library Management System ===
1. Show all books
2. Show all users
3. Show borrow records
4. Borrow a book
5. Return a book
6. Exit



Enter your choice (1-6):  4
Enter user ID:  U1001
Enter book ID:  D2889


Borrowing failed: You have reached the borrowing limit of books!!

=== Library Management System ===
1. Show all books
2. Show all users
3. Show borrow records
4. Borrow a book
5. Return a book
6. Exit



Enter your choice (1-6):  5
Enter user ID:  U1002
